In [1]:
from transformers import pipeline, AutoTokenizer, AutoModel, BloomForCausalLM, BloomForQuestionAnswering, Conversation, AutoModelForCausalLM
import re
import torch

/home/kyle/anaconda3/envs/npcenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# MODELS DEMO FOR NPC CAPSTONE

Peek into my brain to help with creating a proposal for GWU Capstone. Purpose of this notebook it to explore the functionality of different models within huggingface and see how they could potentially fit
within our use case. Point is NOT TO: Use the best models (Tried to keep it small), optimize for performance (These should all be set up to run on GPU).

TAKEAWAYS:

This notebook helped created the idea for the following pipeline:

1. **Context generating model**: creates the backstory for a character that gets fed into the conversational model. Question answering could maybe be leveraged to come up with stats or traits as well. This would probably be a fine-tuned version of Bloom.
1. **Conversational Model**: Uses the context generated above to generate the actual interactions, still need to look into this but the medium article is a good start.
1. **Decision Model**: Maybe text classification with a large array of possible decision points for each NPC to be able to make. This would get fed in the context from both steps 1 and 2 to determine something like: This is an item being sold, this is an npc attacking the player, this is the npc healing the player, etc. NER could exist as well for pulling out items and such


In [2]:
# Helpers

def clean_output_generator(output):
    """
    Takes the output of our generator model and returns it in a clean, more readable format.
    """
    return(re.sub("\\'","'",re.sub(" +", " ",output.replace("\n"," "))))

In [3]:
# We can specify different models and tokenizers if we'd like, and they dont have to match. For now using smallest bloom
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")

generator = pipeline(model = "bigscience/bloom-560m")

In [4]:
# Text generation pipeline
generator.check_model_type

<bound method Pipeline.check_model_type of <transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f9c52019960>>

## Text Generation

Text generation seems to be the completion of stories, conversations, code, and other forms of text. This is what Bloom does by default, so not sure if it has conversational capabilities. However, this could still be very useful for filling in context, completing a conversation, or developing a character biography.

In [5]:
example = """Carlos is a cowboy from the midwest. 
        He comes from a large family with 5 brothers and 4 sisters. He enjoys reading, telling stories,
        and going hunting. Some of his biggest strengths as a survivalist are
        """
clean_output_generator(generator(example, max_length = 75)[0]['generated_text'])

'Carlos is a cowboy from the midwest. He comes from a large family with 5 brothers and 4 sisters. He enjoys reading, telling stories, and going hunting. Some of his biggest strengths as a survivalist are his ability to survive in the wilderness and his ability to fight back. He is a very good friend and a good person. He is'

In [6]:
## Some examples show the pipeline broken into pieces like this, but not sure what the benefit is. 
# Maybe extra control over parameters.
example = """Carlos is a cowboy from the midwest. 
        He comes from a large family with 5 brothers and 4 sisters. He enjoys reading, telling stories,
        and going hunting. Some of his biggest strengths as a survivalist are
        """
inputs = tokenizer.encode(example, return_tensors="pt")
outputs = model.generate(inputs, max_length = 75)
clean_output_generator(tokenizer.decode(outputs[0], skip_special_tokens=True))

'Carlos is a cowboy from the midwest. He comes from a large family with 5 brothers and 4 sisters. He enjoys reading, telling stories, and going hunting. Some of his biggest strengths as a survivalist are his ability to survive in the wilderness and his ability to fight back. He is a very good friend and a good person. He is'

## Question Answering

Potentially could be used to fill in context for a character, or assign traits/abilities

In [7]:
# Bloom Model isn't working quite right for QA, looks like it needs to be trained. Im going to try another one.
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b7")
model = BloomForQuestionAnswering.from_pretrained("bigscience/bloom-1b7")

Some weights of BloomForQuestionAnswering were not initialized from the model checkpoint at bigscience/bloom-1b7 and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
question = "Where is Carlos from?"
context = """ Carlos is a cowboy from the midwest. 
        He comes from a large family with 5 brothers and 4 sisters. He enjoys reading, telling stories,
        and going hunting.
        """
inputs = tokenizer(question,context, return_tensors="pt")
with torch.no_grad():
        outputs = model(**inputs)

# Get the highest probability from the model output for the start and end positions:

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)
# clean_output_generator(tokenizer.decode(outputs[0], skip_special_tokens=True))

/home/kyle/anaconda3/envs/npcenv/lib/python3.10/site-packages/transformers/models/bloom/modeling_bloom.py:693: FutureWarning: `position_ids` have no functionality in BLOOM and will be removed in v5.0.0. You can safely ignore passing `position_ids`.
  warnings.warn(


''

In [9]:
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

In [10]:
# Lets try Distilbert for POC
question = "What would be Carlos's Hunting stat out of 10?"
context = """ Carlos is a cowboy from the midwest. 
        He comes from a large family with 5 brothers and 4 sisters. He enjoys reading, telling stories,
        and going hunting. He is a very skilled hunter.
        """

result = question_answerer(question,context)
print(question)
print(
f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

# I thought this use case might work, but I don't think the QA models are Generative so maybe not. Maybe this
# second use case is applicable, where we use the train the generative model from above to output with this
# in mind and then use Question Answering to extract from the context

question = "What are Carlos's traits?"
context = """ Carlos is a cowboy from the midwest. He is an experienced hunter so he is equipped with
        the following traits: hunter and gatherer, survivalist, and sharpshooter. 
        """

result = question_answerer(question,context)
print(question)
print(
f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

What would be Carlos's Hunting stat out of 10?
Answer: 'very skilled hunter', score: 0.1612, start: 178, end: 197
What are Carlos's traits?
Answer: 'hunter and gatherer, survivalist, and sharpshooter', score: 0.8857, start: 119, end: 169


## Conversational Model 

This is what the user will directly interact with. It will take context created from the generative model and use it to build
the character. Curious if we can incorporate things like character traits, stats, or equipment so it knows to tie it into the conversation.

https://medium.com/huggingface/how-to-build-a-state-of-the-art-conversational-ai-with-transfer-learning-2d818ac26313

### Concerns

Getting the model to abide by the rules of the world the character exists within.  

In [11]:
tokenizer = AutoTokenizer.from_pretrained("PygmalionAI/pygmalion-1.3b")
model = AutoModelForCausalLM.from_pretrained("PygmalionAI/pygmalion-1.3b")

In [12]:
#Stopping Criteria is require to exit after the Bot's response
from transformers import StoppingCriteriaList, StoppingCriteria
class _SentinelTokenStoppingCriteria(StoppingCriteria):

    def __init__(self, sentinel_token_ids: torch.LongTensor,
                 starting_idx: int):
        StoppingCriteria.__init__(self)
        self.sentinel_token_ids = sentinel_token_ids
        self.starting_idx = starting_idx

    def __call__(self, input_ids: torch.LongTensor,
                 _scores: torch.FloatTensor) -> bool:
        for sample in input_ids:
            trimmed_sample = sample[self.starting_idx:]
            # Can't unfold, output is still too tiny. Skip.
            if trimmed_sample.shape[-1] < self.sentinel_token_ids.shape[-1]:
                continue

            for window in trimmed_sample.unfold(
                    0, self.sentinel_token_ids.shape[-1], 1):
                if torch.all(torch.eq(self.sentinel_token_ids, window)):
                    return True
        return False


In [13]:
# For this model, they recommend the input take on the following format. I'll copy over the same context as before to keep things uniform
conversation = """
Carlos's Persona:  Carlos is a cowboy from the midwest. 
        He comes from a large family with 5 brothers and 4 sisters. 
        He enjoys reading, telling stories,
        and going hunting. 
        He is a very skilled hunter. 
        He is untrustworthy of strangers.
<START>
You: Hi Carlos, care to join me on a hunting trip?
Carlos:
"""
tokenized_items = tokenizer(conversation, return_tensors="pt")

stopping_criteria_list = StoppingCriteriaList([
        _SentinelTokenStoppingCriteria(
                sentinel_token_ids=tokenizer(
                "\nYou:",
                add_special_tokens=False,
                return_tensors="pt",
                ).input_ids,
                starting_idx=tokenized_items.input_ids.shape[-1])
])


logits = model.generate(stopping_criteria=stopping_criteria_list, **tokenized_items, max_new_tokens=200)
print(clean_output_generator(tokenizer.decode(logits[0], skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 Carlos's Persona: Carlos is a cowboy from the midwest. He comes from a large family with 5 brothers and 4 sisters. He enjoys reading, telling stories, and going hunting. He is a very skilled hunter. He is untrustworthy of strangers. <START> You: Hi Carlos, care to join me on a hunting trip? Carlos: "Me? I'm afraid I'm a bit of a loner. I've had the same group of friends for my whole life. I'm afraid to show you my true intentions." You:


In [14]:
# I am adding another example for the case of a merchant selling goods as this works for the Decision model I chose as a poc.
conversation = """
Scenario: You are trying to purchase something from Carlos's shop. The ruby you are trying to buy is for sale for 10 gold.
Carlos's Persona:  Carlos is a merchant.
    Carlos sells jewels out of his shop.
    Carlos will sell his jewels for cheap.
<START>
You: I am looking to buy a ruby.
Carlos: The rubies I sell cost 10 gold.
You: I am willing to pay 10 gold, do we have a deal?
Carlos:
"""
tokenized_items = tokenizer(conversation, return_tensors="pt")

stopping_criteria_list = StoppingCriteriaList([
        _SentinelTokenStoppingCriteria(
                sentinel_token_ids=tokenizer(
                "\nYou:",
                add_special_tokens=False,
                return_tensors="pt",
                ).input_ids,
                starting_idx=tokenized_items.input_ids.shape[-1])
])


logits = model.generate(stopping_criteria=stopping_criteria_list, **tokenized_items, max_new_tokens=200)
print(clean_output_generator(tokenizer.decode(logits[0], skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 Scenario: You are trying to purchase something from Carlos's shop. The ruby you are trying to buy is for sale for 10 gold. Carlos's Persona: Carlos is a merchant. Carlos sells jewels out of his shop. Carlos will sell his jewels for cheap. <START> You: I am looking to buy a ruby. Carlos: The rubies I sell cost 10 gold. You: I am willing to pay 10 gold, do we have a deal? Carlos: *Carlos looks at you* We do, my friend. You:


## Decision Model

The one that I have no idea if it'll work. Should take the conversation, scenario, and persona and decide what kind of action is being performed. For example: 

*Hi Carlos, care to join me on a hunting trip?* -> gets classified as **request to join party**

*Me? I'm afraid I'm a bit of a loner. I've had the same group of friends for my whole life. I'm afraid to show you my true intentions.* -> gets classified as **denial of request to join party**

I image a couple methods for doing this, text classification may be fine but topic modelling could be better.

I have found the following model (`j-hartmann/purchase-intention-english-roberta-large`) which classifies if an "expressed purchase intention" so I am using this as a proof of concept

### Dialogue Act Classification

This seems like it could be an important step in distinguishing the intent of the users input. It is able to classify sentences based on their intent within Dialogue. Examples include:

- Greeting
- Opinion
- Y/N Question

and so on... Couldn't yet find a model, but found this (https://paperswithcode.com/sota/dialogue-act-classification-on-switchboard). None of these models use transformers, so they are not hosted on huggingface. However we could reproduce them locally if we would like as all data and model parameters are posted    

## Intent Classification

This seems like a more appropriate solution. Simply create a text classification model to classify a conversation's text into an outcome? I'm thinking something like how Amazon Alexa is able to perform actions based on your speech input. Would be a challenge to really include a plethora of intents. Worth talking about

In [15]:
classifier = pipeline("text-classification", model="j-hartmann/purchase-intention-english-roberta-large", return_all_scores=True)

Some weights of the model checkpoint at j-hartmann/purchase-intention-english-roberta-large were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/kyle/anaconda3/envs/npcenv/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_

In [16]:
# Here the players input is classified as an "Expressed purchase intention", thus we can look at the NPC's output to see if a deal was accepted or not
classifier("I am willing to pay 10 gold, do we have a deal?")

[[{'label': 'no', 'score': 0.01472777035087347},
  {'label': 'yes', 'score': 0.985272228717804}]]

## Bloom for the other purposes

Can we use bloom to solve these other problems as well

In [18]:
example = """
        Respond as if you are Charles, a salesman from the midwest. The currency in this world is gold and Charles sells healing herbs, health potions, and strength potions.
        Charles currently has 10 of each of these items in stock, and they cost 1, 5, and 6 gold respectively.
        James: I would like to buy one health potion
        Charles:
          """

clean_output_generator(generator(example, max_length = 150)[0]['generated_text'])

' Respond as if you are Charles, a salesman from the midwest. The currency in this world is gold and Charles sells healing herbs, health potions, and strength potions. Charles currently has 10 of each of these items in stock, and they cost 1, 5, and 6 gold respectively. James: I would like to buy one health potion Charles: 1. A healing potion James: 2. A strength potion James: 3. A healing potion James: 4. A strength potion James: 5. A healing potion James: 6. A healing potion James: 7. A healing potion James: 8.'